# #DataMadness

In [46]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv("hydra_movie_scrape.csv")
data.head()

#First we will see if we have some movies twice in our data frame
data = data.drop_duplicates(subset=['title'])


#From the data count we see that they are missing some vallues 


data = data.dropna(subset=['cast'])
data = data.dropna(subset=['writers'])
data = data.dropna(subset=['summary'])
data = data.dropna(subset=['short_summary'])

#we drop the collum with that contains the link for the youtube trailer for each of the movie
data = data.drop(columns=['youtube_trailer'])
print(data.count())

print(sum(data['genres'] == 'Uncategorized'))
data = data[data['runtime'] != 0]
data = data[data['genres'] !='Uncategorized']

print(data.count())
#problem with genres multiple options 

#def ohe_genres(data):
  
#    cats = [
#        'Action',
#        'Documentary|Music',
#        'Adventure',
#        'Animation',
#        'Family',
#        'Fantasy'
#    ]
    
#   cat_type = CategoricalDtype(categories=cats)
    
#    data['genres'] = data['genres'].astype(cat_type)
#   data = pd.get_dummies(data,
#                          prefix='fpq',
#                          columns=['genres'], 
#                         drop_first=True)
    
#    return data
#data = ohe_genres(data)
#data.filter(regex='fpq').head(10)


#split genres , max number of genres for each movie is 5 

genres_split = data['genres'].str.split('[,|]', expand=True)

genres_split.columns = ['genre1', 'genre2', 'genre3', 'genre4', 'genre5']
genres_split.head(10)

title            3889
year             3889
summary          3889
short_summary    3889
genres           3889
imdb_id          3889
runtime          3889
rating           3889
movie_poster     3889
director         3889
writers          3889
cast             3889
dtype: int64
25
title            3684
year             3684
summary          3684
short_summary    3684
genres           3684
imdb_id          3684
runtime          3684
rating           3684
movie_poster     3684
director         3684
writers          3684
cast             3684
dtype: int64


,genre1,genre2,genre3,genre4,genre5
1,Documentary,Music,None,None,None
2,Adventure,Animation,Comedy,Family,Fantasy
4,Drama,None,None,None,None
5,Animation,Comedy,Family,Sport,None
6,Adventure,Animation,Comedy,Family,Fantasy
9,Drama,None,None,None,None
10,Drama,Music,Romance,None,None
14,Comedy,Drama,None,None,None
15,Action,Animation,None,None,None
16,Fantasy,Horror,None,None,None
